In [1]:
import pandas as pd
import os
import sys
sys.path.insert(1, "../data")
sys.path.insert(2, "../scripts")
from constants import *
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
import math

#### Constants

In [2]:
str_to_replace = {
    "mercedes0benz": "mercedes-benz",
    "vw": "volkswagen"}

### Data Imports

In [3]:
external_sales = pd.read_csv(PREFIX + CURATED_DIR + "/salesbycarbrand.csv")
external_sales[MAKER] = external_sales[MAKER].str.lower()
external_sales[MAKER] = external_sales[MAKER].replace(str_to_replace)

dealer_sales = pd.read_csv(PREFIX+CURATED_DIR+"/car_prices_curated.csv").drop(['Unnamed: 0', "vin", "state"], axis=1)
dealer_sales["make"] = dealer_sales["make"].str.lower()
cars_dealt = sorted(dealer_sales["make"].unique())

In [4]:
joined_df = dealer_sales.merge(external_sales, left_on="make", right_on="maker").drop(["maker"], axis=1)

### Encoding + Dimension Reduction

In [5]:
joined_df.columns
cat_cols = ["trim", "body", "transmission", "state_name", "make", "sale_month", "model"]``

In [6]:
# perform one-hot encoding on the 'color' column
for col in cat_cols:
    
    one_hot = pd.get_dummies(joined_df[col], dtype=int)

    joined_df = pd.concat([joined_df, one_hot], axis=1).drop(col, axis=1)

In [7]:
truth_labels = joined_df["sellingprice"]
joined_df = joined_df.drop(["sellingprice"], axis=1)

#### Train-test Split

In [8]:
train_df, test_df, y_train, y_test = train_test_split(joined_df, truth_labels, random_state=42, train_size=0.9)

In [9]:
features_name = list(train_df.columns)
feature_scores = mutual_info_regression(train_df, y_train, random_state=42)
proportion = math.ceil((len(features_name))*0.01)
name_score_zip = sorted(list(zip(features_name, feature_scores)), key=lambda x:x[1], reverse=True)[0:proportion]
selected_features = [x[0] for x in name_score_zip]

ValueError: could not convert string to float: 'Rogue'

In [12]:
train_df

,year,model,condition,odometer,color,interior,seller,mmr,saledate,sale_year,...,volkswagen,volvo,1,2,3,4,5,6,7,12
132756,2013,Rogue,39,60448,red,black,the hertz corporation,12950,2015-02-24,2015,...,0,0,0,1,0,0,0,0,0,0
466080,2012,2500,1,107636,white,black,enterprise vehicle exchange / tra / rental / t...,16400,2015-05-20,2015,...,0,0,0,0,0,0,1,0,0,0
368487,2007,Patriot,27,122891,black,gray,santander consumer,4775,2015-02-19,2015,...,0,0,0,1,0,0,0,0,0,0
471136,2014,Town and Country,43,43814,white,black,avis corporation,20200,2015-02-10,2015,...,0,0,0,1,0,0,0,0,0,0
180467,2013,Tacoma,49,15875,gray,gray,allan a hornbeck chevrolet sales inc,27900,2015-06-05,2015,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,2012,Volt,29,80607,gray,black,ge fleet services for itself/servicer,9925,2015-03-06,2015,...,0,0,0,0,1,0,0,0,0,0
365838,2011,Wrangler,41,28449,black,black,tdaf remarketing,25800,2015-01-06,2015,...,0,0,1,0,0,0,0,0,0,0
131932,2014,Sentra,4,22394,silver,black,southeastern car & truck rental,12050,2015-01-20,2015,...,0,0,1,0,0,0,0,0,0,0
146867,2013,E-Class,26,41284,black,black,the hertz corporation,30700,2015-03-03,2015,...,0,0,0,0,1,0,0,0,0,0


In [11]:
scaler = StandardScaler()
train_df = scaler.fit_transform(train_df)
test_df = scaler.transform(test_df)
pca = PCA(n_components=20)
pca

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.